In [1]:
import pandas as pd
import re

# import csv files as Pandas dataframes

In [2]:
# geologiclog_USCS ; released by Ben Brezing on OpenData/OSWCR 09/26/2024
df = pd.read_csv("geologiclog_uscs.csv", encoding='utf-8-sig')

df.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,SOILCLASSIFICATION,SOILCOLOR,SOILDESCRIPTION
0,WCR2019-015573,7.0,10.0,SW Well-graded SAND,NaN,NaN
1,WCR2019-015573,10.0,13.0,SW Well-graded SAND,NaN,NaN
2,WCR2019-015573,13.0,25.0,SC Clayey SAND,NaN,NaN
3,WCR2019-015573,25.0,26.0,ML Inorganic SILT with low plasticity,NaN,NaN
4,WCR2019-015573,26.0,28.0,SC Clayey SAND,NaN,NaN


In [3]:
# Function to extract text within parentheses
def extract_uscs(text):
    if isinstance(text, str):
        match = re.search(r'\((.*?)\)', text)
        if match:
            return match.group(1)
    return 'unknown'

# Apply the function to the 'SOILCLASSIFICATION' column and create a new 'USCS' column
df['USCS'] = df['SOILCLASSIFICATION'].apply(extract_uscs)

In [4]:
# Display the DataFrame to verify the new column
df.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,SOILCLASSIFICATION,SOILCOLOR,SOILDESCRIPTION,USCS
0,WCR2019-015573,7.0,10.0,SW Well-graded SAND,NaN,NaN,unknown
1,WCR2019-015573,10.0,13.0,SW Well-graded SAND,NaN,NaN,unknown
2,WCR2019-015573,13.0,25.0,SC Clayey SAND,NaN,NaN,unknown
3,WCR2019-015573,25.0,26.0,ML Inorganic SILT with low plasticity,NaN,NaN,unknown
4,WCR2019-015573,26.0,28.0,SC Clayey SAND,NaN,NaN,unknown


In [5]:
# Set of known USCS classifications
known_uscs = {
    'ML', 'CH', 'CL', 'MH', 'SC', 'SM', 'SP', 'SW', 'GW', 'GP', 'GM', 'GC', 'OH',
    'SM-SC', 'SM-ML', 'SM-GP', 'SP-GP', 'SW-GW', 'SC-GC', 'SC-ML', 'SP/GP/CL', 'CL/SP',
    'GW-GM', 'OH/CH', 'CL/CH', 'Cl', 'SW-SM', 'SW-SC', 'SP-SM', 'SP-SC', 'OL', 'GW-GC', 'GP-SP',
    'GP-GM', 'GP-GC', 'GP-CG', 'CL-ML', 'CH+GP',
    # New USCS classifications to be added
    'ASH', 'SM-SW', 'ML/GW', 'SP/GP', 'CH/ML', 'CL/ML', 'SC', 'CL/SC', 'SM/SC', 'GP/SP/CL',
    'GP/GC', 'SP/SC', 'SP/CH', 'GP/SP', 'CH/SP', 'GP/CL'
}

In [6]:
# Dictionary to map specific soil classifications to USCS or rock categories
classification_mapping = {
    'Rock - Sedimentary': 'ROCK',
    'Rock - Igneous': 'IGNS',
    'Rock - Metamorphic': 'META',
    'Siltstone': 'STST',
    'Sandstone': 'SDST',
    
    'Bedrock': 'ROCK',
    'SILTSTONE/MUDSTONE': 'STST,CLSN',
    'Topsoil': 'OH',
    'FILL': 'unknown',
    'Claystone/hardpan': 'CLSN,ML',
    'Siltstone-Claystone': 'STST,CLSN',
    'Silty-Sandstone': 'SDST,STST',
    'Ishi': 'VOLC',
    'Ash': 'ASH',
    'Tuff': 'TUFF',
    'sltst': 'STST',
    'clyst': 'CLSN',
    'Mudstone': 'CLSN',
    'sltst and clyst': 'STST,CLSN',
    'CLAYSTONE/SAND': 'CLSN,SC',
    'CLAYSTONE': 'CLSN',
    'TUFF or SILTSTONE/CLAYSTONE': 'TUFF,STST,CLSN',
    'SILTSTONE/SANDSTONE': 'STST,SDST',
    'SIlTSTONE/CLAYSTONE': 'STST,CLSN',
    'Sand/Sandstone': 'SDST,SW',
    'Volcanic': 'volcanic rock',
    'PT PEAT soils with high organic contents': 'PT',
    'Claystone/hardpan,': 'CLSN,ML',
    'Siltstone-Claystone,': 'STST,CLSN',
    'Silty-Sandstone,': 'SDST, ML',
    'Basalt': 'BSLT',
    'Sltst': 'STST',
    'SILTSTONE':'STST',
    'SANDSTONE':'SDST',
    'Sandstone/Siltstone':'SDST,STST',
    'SANDSTONE/SC':'SDST,SC',
    'MUDSTONE':'CLSN',
    'BASALT':'BSLT',
    'CONGLOMERATE':'CONG',
    'GP/CH':'GP,CH',
    'Claystone':'CLSN',
    'CLAYSTONE/MUDSTONE':'CLSN',
    'TUFF':'TUFF',
    'Top soil':'TPSL',
    'SILTSTONE/SANDSTONE,':'STST,SDST',
    'SilTSTONE/CLAYSTONE':'STST,CLSN',
    'GM/SP':'GM,SP',
    'Tuff/ASH':'TUFF, ASH',

        # Including the ones with BOM characters manually removed
    'ï»¿GC': 'GC',
    'ï»¿SP': 'SP',
    'ï»¿CL': 'CL',

}

In [7]:
# Function to extract USCS classification
def extract_uscs(text):
    if isinstance(text, str):
        text = text.strip()  # Remove leading/trailing whitespace
        # Remove BOM characters if present
        text = text.encode('utf-8').decode('utf-8-sig')
        # Check for direct USCS classification
        for uscs in known_uscs:
            if uscs in text.split():
                return uscs
        # Check for known classifications in the mapping
        if text in classification_mapping:
            return classification_mapping[text]
        # If no direct classification, look for text within parentheses
        match = re.search(r'\((.*?)\)', text)
        if match:
            return match.group(1)
    return 'unknown'

# Apply the function to the 'SOILCLASSIFICATION' column and create a new 'USCS' column
df['USCS'] = df['SOILCLASSIFICATION'].apply(extract_uscs)

# Display the DataFrame to verify the new column
df.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,SOILCLASSIFICATION,SOILCOLOR,SOILDESCRIPTION,USCS
0,WCR2019-015573,7.0,10.0,SW Well-graded SAND,NaN,NaN,SW
1,WCR2019-015573,10.0,13.0,SW Well-graded SAND,NaN,NaN,SW
2,WCR2019-015573,13.0,25.0,SC Clayey SAND,NaN,NaN,SC
3,WCR2019-015573,25.0,26.0,ML Inorganic SILT with low plasticity,NaN,NaN,ML
4,WCR2019-015573,26.0,28.0,SC Clayey SAND,NaN,NaN,SC


In [9]:
# Load the provided Excel file
file_path = r'C:\Users\betebari\Documents\C2VSim_Texture\OSWCR\USCS-averageKxy-CoarseFractions.xlsx'
excel_data = pd.read_excel(file_path)

# Strip any leading/trailing spaces from 'Sediment/Rock Type' column in the Excel data
excel_data['Sediment/Rock Type'] = excel_data['Sediment/Rock Type'].str.strip()

# Convert 'Sediment/Rock Type' to lowercase for case-insensitive matching
excel_data['Sediment/Rock Type'] = excel_data['Sediment/Rock Type'].str.lower()

# Create a dictionary mapping Soil Classification to Average Hydraulic Conductivity (case-insensitive)
hydraulic_conductivity_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Average Hydraulic Conductivity (ft/day)']))

# Create a dictionary mapping Soil Classification to Average Coarse Fraction (case-insensitive)
coarse_fraction_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Average Coarse Fraction (%)']))

# Strip any leading/trailing spaces from 'USCS' column in the merged DataFrame
df['USCS'] = df['USCS'].str.strip()

# Remove parentheses from USCS values
df['USCS'] = df['USCS'].str.replace(r'[\(\)]', '', regex=True)

# Convert 'USCS' column to lowercase for case-insensitive matching
df['USCS'] = df['USCS'].str.lower()

# Create a dictionary mapping Soil Classification to Average Hydraulic Conductivity
hydraulic_conductivity_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Average Hydraulic Conductivity (ft/day)']))

# Create a dictionary mapping Soil Classification to Average Coarse Fraction
coarse_fraction_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Average Coarse Fraction (%)']))

# Create separate mappings for Specific Yield, Ss, and Kv based on the Excel data columns
specific_yield_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Avg Specific Yield (%)']))
ss_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Avg Ss (1/L)']))
kv_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Avg Kv (ft/day)']))

# Corrected function to handle more complex USCS combinations
def aggregate_uscs_values(uscs_value, mapping, agg_func='average'):
    # Handle 'unknown' case by returning None or a default value
    if 'unknown' in uscs_value:
        return None  # You can replace None with a default value like 0 or 12 depending on your needs

    # Corrected regex to handle different separators
    uscs_list = re.split(r'[-\s,/+]+', uscs_value.lower().strip())
    
    values = [mapping.get(uscs.strip()) for uscs in uscs_list if uscs in mapping]
    
    # Apply 12% coarse fraction if secondary USCS classification is present and missing from mapping
    if 'gc' in uscs_list or 'sc' in uscs_list or 'gm' in uscs_list or 'sm' in uscs_list:
        if agg_func == 'average':
            # Assign 12% when no data is found for the given classification
            return sum(values) / len(values) if values else 12
        elif agg_func == 'max':
            return max(values) if values else 12
    
    if values:
        if agg_func == 'average':
            return sum(values) / len(values)
        elif agg_func == 'max':
            return max(values)
    
    return None

# Apply the enhanced function for Hydraulic Conductivity and Coarse Fraction
df['HydraulicConductivity'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, hydraulic_conductivity_mapping, agg_func='average'))
df['AverageCoarseFraction'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, coarse_fraction_mapping, agg_func='average'))
df['Avg Specific Yield (%)'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, specific_yield_mapping, agg_func='average'))
df['Avg Ss (1/L)'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, ss_mapping, agg_func='average'))
df['Avg Kv (ft/day)'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, kv_mapping, agg_func='average'))

# Check for remaining unmatched values
unmatched_values = df[df['HydraulicConductivity'].isna()]['USCS'].unique()
print("Remaining unmatched 'USCS' values:", unmatched_values)

Remaining unmatched 'USCS' values: ['unknown']


In [10]:
df1 = pd.read_csv("UPDATED_wellcompletionreports.csv")
# Merge df1 and df2 on 'WCRNUMBER'
merged_df = pd.merge(df1, df, on='WCRNUMBER', how='inner')

# Drop rows where 'DESCRIPTION' is empty (NaN or empty string)
merged_df = merged_df[merged_df['INTERVALSTART'].notna() & (merged_df['INTERVALSTART'] != '')]

# Convert the 'USCS' column to uppercase
merged_df['USCS'] = merged_df['USCS'].str.upper()

# Display the DataFrame
merged_df.head()

,Unnamed: 0,WCRNUMBER,DECIMALLAT,DECIMALLON,UTMX_y,UTMY_y,INTERVALSTART,INTERVALEND,SOILCLASSIFICATION,SOILCOLOR,SOILDESCRIPTION,USCS,HydraulicConductivity,AverageCoarseFraction,Avg Specific Yield (%),Avg Ss (1/L),Avg Kv (ft/day)
0,1871,WCR2018-004813,35.2014,-118.923,871225.039877,3.902999e+06,0.0,7.0,SM Silty SAND,med to dk yellow brown,"Silty Sand - vfn gr sand, damp",SM,9.110200,45.0,15.0,0.000505,0.0500
1,1871,WCR2018-004813,35.2014,-118.923,871225.039877,3.902999e+06,7.0,30.0,SW Well-graded SAND,lt-med yellow brown,"Sand - fn to v crs gr, trace gravel to 1/8""",SW,4.510000,60.0,35.0,0.000230,0.5000
2,1871,WCR2018-004813,35.2014,-118.923,871225.039877,3.902999e+06,30.0,37.0,CL Lean inorganic CLAY with low plasticity,olive brown,"Clay - moist, tr sand, low plast",CL,0.016402,5.0,3.0,0.002500,0.0005
3,1871,WCR2018-004813,35.2014,-118.923,871225.039877,3.902999e+06,37.0,69.0,SW Well-graded SAND,lt - med yellow brown,"Sand - lt-med yellow brown, fn - v crs gr, som...",SW,4.510000,60.0,35.0,0.000230,0.5000
4,1871,WCR2018-004813,35.2014,-118.923,871225.039877,3.902999e+06,69.0,71.0,GW-GM Well-graded GRAVEL with silt,lt-med brown,Gravelly interval,GW-GM,84.588050,62.5,19.0,0.000278,0.2750


In [11]:
# Save the updated DataFrame to a new CSV file
output_file = '3-updated_geologiclog_USCS.csv'
merged_df.to_csv(output_file, index=False)

print(f"Updated CSV file saved as '{output_file}'")

Updated CSV file saved as '3-updated_geologiclog_USCS.csv'
